In [1]:
!unzip /kaggle/input/santander-product-recommendation/train_ver2.csv.zip

Archive:  /kaggle/input/santander-product-recommendation/train_ver2.csv.zip
  inflating: train_ver2.csv          


In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
from xgboost import XGBClassifier 
from sklearn.utils.class_weight import compute_class_weight

# Data Preprocessing
***

In [3]:
# load product recommendation database.

df_full = pd.read_csv("/kaggle/working/train_ver2.csv")

<ipython-input-3-07688df9c90b>:3: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv("/kaggle/working/train_ver2.csv")


In [4]:
df_full.head(5)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [5]:
# length of the database.

len(df_full['ncodpers'].unique()), len(df_full)

(956645, 13647309)

In [6]:
# number of customer_ids with loans.

len(df_full[df_full['ind_pres_fin_ult1'] == 1]['ncodpers'].unique()), len(df_full[df_full['ind_pres_fin_ult1'] == 1]['ncodpers'].unique())/len(df_full['ncodpers'].unique())

(2456, 0.0025673055313099424)

In [7]:
# pre-process the dataset.

# (a) combine the following columns: 1. Accounts - Savings + Current + Payroll + Junior + Particular + Mas Particular + Direct Debit + e-account
# 2. Investments - Derivatives Accounts + Funds + Securities 
# 3. Loans - Loans + Mortgage + Home Account 
# 4. Rename Credit Card + Deposits 
df_full['Accounts'] = df_full['ind_ahor_fin_ult1'] | df_full['ind_cco_fin_ult1'] | df_full['ind_cno_fin_ult1'] | df_full['ind_ctju_fin_ult1'] | df_full['ind_ctma_fin_ult1']| df_full['ind_ctop_fin_ult1'] | df_full['ind_ctpp_fin_ult1'] | df_full['ind_ecue_fin_ult1'] | df_full['ind_recibo_ult1']
df_full['Investments'] = df_full['ind_cder_fin_ult1'] | df_full['ind_valo_fin_ult1'] | df_full['ind_fond_fin_ult1']
df_full['Credit Cards'] = df_full['ind_tjcr_fin_ult1']
df_full['Loans'] = df_full['ind_hip_fin_ult1'] | df_full['ind_pres_fin_ult1'] | df_full['ind_viv_fin_ult1']
df_full['Short Term Deposit'] = df_full['ind_deco_fin_ult1']
df_full['Medium Term Deposit'] = df_full['ind_deme_fin_ult1']
df_full['Long Term Deposit'] = df_full['ind_dela_fin_ult1']
df_full['segmento'].replace(to_replace=['02 - PARTICULARES', '03 - UNIVERSITARIO', '01 - TOP'], value=[0, 1, 2], inplace=True)
df_full = df_full[df_full['indfall'] == 'N']
df_full.rename(columns={"ind_empleado":"Employment", "indresi":"Residence", "indext":"Foreigner", "segmento":"Segment", "fecha_dato":"date", "ncodpers":"Customer code", "sexo":"Gender", "renta":"Income", "ind_actividad_cliente":"Active", "ind_plan_fin_ult1":"Pension"}, inplace=True)
df_full.drop(columns=['fecha_alta','pais_residencia','antiguedad','indrel','ult_fec_cli_1t','indrel_1mes','tiprel_1mes','conyuemp','canal_entrada','ind_nuevo',
                     'cod_prov','nomprov','ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1',
                     'ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1',
                     'ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1'
                     ,'tipodom','indfall'], inplace=True)
df_full.head(5)

<ipython-input-7-dbdacf8813ee>:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full['segmento'].replace(to_replace=['02 - PARTICULARES', '03 - UNIVERSITARIO', '01 - TOP'], value=[0, 1, 2], inplace=True)
<ipython-input-7-dbdacf8813ee>:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_full['se

,date,Customer code,Employment,Gender,age,Residence,Foreigner,Active,Income,Segment,Pension,Accounts,Investments,Credit Cards,Loans,Short Term Deposit,Medium Term Deposit,Long Term Deposit
0,2015-01-28,1375586,N,H,35,S,N,1.0,87218.10,0.0,0,1,0,0,0,0,0,0
1,2015-01-28,1050611,N,V,23,S,S,0.0,35548.74,1.0,0,1,0,0,0,0,0,0
2,2015-01-28,1050612,N,V,23,S,N,0.0,122179.11,1.0,0,1,0,0,0,0,0,0
3,2015-01-28,1050613,N,H,22,S,N,0.0,119775.54,1.0,0,0,0,0,0,1,0,0
4,2015-01-28,1050614,N,V,23,S,N,1.0,NaN,1.0,0,1,0,0,0,0,0,0


In [8]:
# make the pivot.

pivot_df = df_full.sort_values('date',ascending=False).groupby("Customer code").agg({'Employment':'last','Gender':'last','age':'last','Residence':'last','Foreigner':'last','Active':'last',
                                                                                     'Pension':'last','Income':'last','Segment':'last','Accounts':'any','Investments':'any',
                                                                                     'Credit Cards':'any','Loans':'any','Short Term Deposit':'any','Medium Term Deposit':'any',
                                                                                     'Long Term Deposit':'any'}).reset_index()
pivot_df[['Pension','Accounts','Investments','Credit Cards','Loans','Short Term Deposit','Medium Term Deposit','Long Term Deposit']] = pivot_df[['Pension','Accounts','Investments','Credit Cards','Loans','Short Term Deposit','Medium Term Deposit','Long Term Deposit']].astype('float32')
pivot_df = pivot_df.fillna(value={'Segment':0.0, 'Income':pivot_df['Income'].median()})
scaler = StandardScaler()
age_scaler = MinMaxScaler()
scaler.fit(pivot_df['Income'].to_numpy().reshape((-1,1)))
age_scaler.fit(pivot_df['age'].to_numpy().reshape((-1,1)))
pivot_df

,Customer code,Employment,Gender,age,Residence,Foreigner,Active,Pension,Income,Segment,Accounts,Investments,Credit Cards,Loans,Short Term Deposit,Medium Term Deposit,Long Term Deposit
0,15889,F,V,56,S,N,1.0,0.0,326124.90,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,15890,A,V,62,S,N,1.0,1.0,71461.20,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,15891,N,H,59,S,N,1.0,0.0,101440.23,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15892,F,H,61,S,N,1.0,0.0,430477.41,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
4,15893,N,V,62,S,N,1.0,0.0,430477.41,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947365,1553685,N,V,52,S,N,0.0,0.0,101440.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
947366,1553686,N,H,30,S,S,0.0,0.0,101440.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
947367,1553687,N,V,21,S,N,0.0,0.0,101440.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
947368,1553688,N,H,43,S,N,0.0,0.0,101440.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# preprcoessing the full dataframe using value filling from the pivot.

df_full[['Pension','Accounts','Investments','Credit Cards','Loans','Short Term Deposit','Medium Term Deposit','Long Term Deposit']] = df_full[['Pension','Accounts','Investments','Credit Cards','Loans','Short Term Deposit','Medium Term Deposit','Long Term Deposit']].astype('float32')
df_full = df_full.fillna(value={'Segment':0.0, 'Income':pivot_df['Income'].median()})
df_full = pd.get_dummies(df_full, columns=['Employment','Gender','Residence','Foreigner','Segment'], dtype='float32')
df_full['Income'] = scaler.transform(df_full['Income'].to_numpy().reshape((-1,1)))
df_full['age'] = age_scaler.transform(df_full['age'].to_numpy().reshape((-1,1)))
df_full

,date,Customer code,age,Active,Income,Pension,Accounts,Investments,Credit Cards,Loans,...,Employment_S,Gender_H,Gender_V,Residence_N,Residence_S,Foreigner_N,Foreigner_S,Segment_0.0,Segment_1.0,Segment_2.0
0,2015-01-28,1375586,0.204969,1.0,-0.191628,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
1,2015-01-28,1050611,0.130435,0.0,-0.448586,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,2015-01-28,1050612,0.130435,0.0,-0.017762,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,2015-01-28,1050613,0.124224,0.0,-0.029715,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,2015-01-28,1050614,0.130435,1.0,-0.120899,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13647304,2016-05-28,1166765,0.124224,0.0,-0.406994,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
13647305,2016-05-28,1166764,0.130435,0.0,-0.509327,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
13647306,2016-05-28,1166763,0.279503,1.0,-0.120899,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
13647307,2016-05-28,1166789,0.124224,0.0,0.367227,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [10]:
# training data for the recommendation model. 

Y_df = df_full[['Accounts','Investments','Credit Cards','Loans','Short Term Deposit','Medium Term Deposit','Long Term Deposit']]
X_df = df_full.drop(columns=['Accounts','Investments','Credit Cards','Loans','Short Term Deposit','Medium Term Deposit','Long Term Deposit'])
X_df

,date,Customer code,age,Active,Income,Pension,Employment_A,Employment_B,Employment_F,Employment_N,Employment_S,Gender_H,Gender_V,Residence_N,Residence_S,Foreigner_N,Foreigner_S,Segment_0.0,Segment_1.0,Segment_2.0
0,2015-01-28,1375586,0.204969,1.0,-0.191628,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
1,2015-01-28,1050611,0.130435,0.0,-0.448586,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,2015-01-28,1050612,0.130435,0.0,-0.017762,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,2015-01-28,1050613,0.124224,0.0,-0.029715,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,2015-01-28,1050614,0.130435,1.0,-0.120899,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13647304,2016-05-28,1166765,0.124224,0.0,-0.406994,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
13647305,2016-05-28,1166764,0.130435,0.0,-0.509327,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
13647306,2016-05-28,1166763,0.279503,1.0,-0.120899,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
13647307,2016-05-28,1166789,0.124224,0.0,0.367227,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [11]:
# viewing Y_df.

Y_df

,Accounts,Investments,Credit Cards,Loans,Short Term Deposit,Medium Term Deposit,Long Term Deposit
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
13647304,1.0,0.0,0.0,0.0,0.0,0.0,0.0
13647305,1.0,0.0,0.0,0.0,0.0,0.0,0.0
13647306,1.0,0.0,0.0,0.0,0.0,0.0,0.0
13647307,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# unique user ids - for IntegerLookup vocabulary. 

unique_user_ids = pivot_df['Customer code'].to_list()

In [13]:
# load loans dataset. 

df_loans = pd.read_csv("/kaggle/input/loan-approval-dataset/Loan Dataset.csv")
df_loans 

,Applicant_ID,Gender,Age,Marital_Status,Dependents,Education,Employment_Status,Occupation_Type,Residential_Status,City/Town,...,Loan_Amount_Requested,Loan_Term,Loan_Purpose,Interest_Rate,Loan_Type,Co-Applicant,Bank_Account_History,Transaction_Frequency,Default_Risk,Loan_Approval_Status
0,1,Female,25,Married,2,Graduate,Employed,Business,Own,Urban,...,24535,209,Home,4.27,Secured,Yes,8,20,0.81,1
1,2,Male,36,Married,2,High School,Employed,Business,Own,Suburban,...,8288,33,Home,14.78,Unsecured,Yes,9,9,0.17,0
2,3,Male,43,Single,0,Postgraduate,Self-Employed,Freelancer,Own,Urban,...,10308,159,Vehicle,12.33,Secured,Yes,7,27,0.25,0
3,4,Female,28,Married,0,High School,Self-Employed,Freelancer,Rent,Suburban,...,33937,39,Personal,8.77,Secured,No,9,16,0.27,1
4,5,Female,32,Single,0,Graduate,Employed,Salaried,Rent,Suburban,...,23360,34,Home,9.04,Unsecured,No,1,17,0.32,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51995,51996,Female,47,Divorced,1,High School,Self-Employed,Professional,Own,Urban,...,15723,60,Home,12.52,Secured,Yes,2,26,0.61,1
51996,51997,Male,25,Married,3,High School,Self-Employed,Freelancer,Rent,Urban,...,21209,237,Vehicle,5.21,Unsecured,No,8,26,0.04,1
51997,51998,Female,48,Married,0,High School,Unemployed,Business,Rent,Urban,...,10540,60,Education,10.60,Unsecured,No,8,24,0.31,0
51998,51999,Female,48,Divorced,1,High School,Employed,Business,Own,Urban,...,16765,32,Vehicle,8.11,Secured,Yes,3,12,0.34,0


In [14]:
# unqiue values for loan purpose - 4 categories of loans (Home, Auto, Education and Personal)

df_loans['Loan_Purpose'].unique()

array(['Home', 'Vehicle', 'Personal', 'Education'], dtype=object)

In [15]:
# columns to keep for the model - removing attributes because we can't reliably assume to possess these as part of properiety data. 

df_loans = df_loans.drop(columns=['City/Town','Monthly_Expenses','Existing_Loans','Total_Existing_Loan_Amount','Outstanding_Debt','Loan_History','Loan_Amount_Requested','Loan_Term',
                                 'Interest_Rate','Loan_Type','Co-Applicant','Default_Risk','Applicant_ID'])

In [16]:
# prepare dataset by scaling, one-hot encoding and separate into 4 datasets - one each per purpose of loan.

df_loans = pd.get_dummies(df_loans, columns=['Gender','Marital_Status','Dependents','Education','Employment_Status','Occupation_Type','Residential_Status'], dtype='float32')
df_home_loans = df_loans[df_loans['Loan_Purpose'] == 'Home'].drop(columns="Loan_Purpose")
df_auto_loans = df_loans[df_loans['Loan_Purpose'] == 'Vehicle'].drop(columns="Loan_Purpose")
df_personal_loans = df_loans[df_loans['Loan_Purpose'] == 'Personal'].drop(columns="Loan_Purpose")
df_edu_loans = df_loans[df_loans['Loan_Purpose'] == 'Education'].drop(columns="Loan_Purpose")
df_loans = df_loans.drop(columns='Loan_Purpose')
loans_scaler = MinMaxScaler()
df_loans = pd.DataFrame(loans_scaler.fit_transform(df_loans), columns=df_loans.columns)
df_home_loans = pd.DataFrame(loans_scaler.transform(df_home_loans), columns=df_loans.columns)
df_auto_loans = pd.DataFrame(loans_scaler.transform(df_auto_loans), columns=df_loans.columns)
df_personal_loans = pd.DataFrame(loans_scaler.transform(df_personal_loans), columns=df_loans.columns)
df_edu_loans = pd.DataFrame(loans_scaler.transform(df_edu_loans), columns=df_loans.columns)
df_edu_loans

,Age,Annual_Income,Credit_Score,Bank_Account_History,Transaction_Frequency,Loan_Approval_Status,Gender_Female,Gender_Male,Marital_Status_Divorced,Marital_Status_Married,...,Employment_Status_Employed,Employment_Status_Self-Employed,Employment_Status_Unemployed,Occupation_Type_Business,Occupation_Type_Freelancer,Occupation_Type_Professional,Occupation_Type_Salaried,Residential_Status_Other,Residential_Status_Own,Residential_Status_Rent
0,0.607843,0.995184,0.897996,0.444444,0.375000,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.705882,0.148882,0.058288,1.000000,0.000000,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.411765,0.728500,0.890710,0.333333,0.916667,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.235294,0.410642,0.948998,0.000000,0.833333,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,0.137255,0.432067,0.823315,0.666667,0.833333,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10339,0.490196,0.399472,0.887067,0.444444,0.375000,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
10340,0.607843,0.642277,0.852459,0.666667,0.916667,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
10341,0.529412,0.998577,0.728597,0.666667,0.000000,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
10342,0.588235,0.288194,0.174863,0.888889,0.791667,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


# Models
***

In [17]:
# loan models.

cfr_t_data = [df_home_loans, df_auto_loans, df_personal_loans, df_edu_loans]
classifiers = []
for cfr_data in cfr_t_data:
    cfr = XGBClassifier()
    cfr.fit(cfr_data.drop(columns='Loan_Approval_Status'), cfr_data['Loan_Approval_Status'])
    print(cfr.score(cfr_data.drop(columns='Loan_Approval_Status'), cfr_data['Loan_Approval_Status']))
    classifiers.append(cfr)

0.8800133849610402
0.916385053586946
0.9255298651252408
0.9222737819025522


In [18]:
for i in range(4):
    model = classifiers[i]
    model.save_model('hackathon_xgboost_model_'+ str(i) +'.model')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [04:49:01] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [19]:
# recommendation model. 

user_id_input = tf.keras.Input(shape=(1,), name="user_id")
user_features_input = tf.keras.Input(shape=(18,), name="user_features")
user_lookup = tf.keras.layers.IntegerLookup(vocabulary=unique_user_ids, mask_token=None)(user_id_input)
user_embeddings = tf.keras.layers.Embedding(len(unique_user_ids)+1, 64)(user_lookup)
user_embeddings = tf.keras.layers.Reshape((64,))(user_embeddings)
concat_features = tf.keras.layers.concatenate([user_features_input, user_embeddings])
layer_1 = tf.keras.layers.Dense(256, activation="relu")(concat_features)
layer_2 = tf.keras.layers.Dense(512, activation="relu")(layer_1)
layer_3 = tf.keras.layers.Dense(64, activation="relu")(layer_2)
output_layer = tf.keras.layers.Dense(7, activation="sigmoid")(layer_3)
model = tf.keras.Model(inputs=[user_id_input, user_features_input], outputs=output_layer)

In [20]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_id (InputLayer)      │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ integer_lookup            │ (None, 1)              │              0 │ user_id[0][0]          │
│ (IntegerLookup)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 1, 64)          │     60,631,744 │ integer_lookup[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_features             │ (None, 18)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 64)             │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 82)             │              0 │ user_features[0][0],   │
│                           │                        │                │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 256)            │         21,248 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 512)            │        131,584 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 64)             │         32,832 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 7)              │            455 │ dense_2[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 60,817,863 (232.00 MB)

 Trainable params: 60,817,863 (232.00 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False))

In [22]:
y_integers = np.argmax(Y_df, axis=1)
class_weights = compute_class_weight('balanced', classes=np.unique(y_integers), y=y_integers)
d_class_weights = dict(enumerate(class_weights))
d_class_weights

{0: 0.14429350167474111,
 1: 62.29336751070718,
 2: 84.22026521844253,
 3: 66.6353375713697,
 4: 289.39570107792593,
 5: 4146.768315018315,
 6: 43.38283121179799}

In [23]:
model.fit([X_df['Customer code'].to_numpy(), X_df.drop(columns=['Customer code','date']).to_numpy()], Y_df.to_numpy(), epochs=1, steps_per_epoch=1_000, class_weight=d_class_weights)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - loss: 2.0368


In [24]:
model.save("hackathon_model.keras")

# Saving Pivot + Synthetic Data 
***

In [25]:
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.1 MB/s eta 0:00:00:00:01


In [73]:
pivot_df

,Customer code,Employment,Gender,age,Residence,Foreigner,Active,Pension,Income,Segment,Accounts,Investments,Credit Cards,Loans,Short Term Deposit,Medium Term Deposit,Long Term Deposit
0,15889,F,Male,56,S,N,1.0,0.0,326124.90,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,15890,A,Male,62,S,N,1.0,1.0,71461.20,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,15891,N,Female,59,S,N,1.0,0.0,101440.23,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15892,F,Female,61,S,N,1.0,0.0,430477.41,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
4,15893,N,Male,62,S,N,1.0,0.0,430477.41,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947365,1553685,N,Male,52,S,N,0.0,0.0,101440.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
947366,1553686,N,Female,30,S,S,0.0,0.0,101440.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
947367,1553687,N,Male,21,S,N,0.0,0.0,101440.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
947368,1553688,N,Female,43,S,N,0.0,0.0,101440.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
df_loans = pd.read_csv("/kaggle/input/loan-approval-dataset/Loan Dataset.csv")
df_loans

,Applicant_ID,Gender,Age,Marital_Status,Dependents,Education,Employment_Status,Occupation_Type,Residential_Status,City/Town,...,Loan_Amount_Requested,Loan_Term,Loan_Purpose,Interest_Rate,Loan_Type,Co-Applicant,Bank_Account_History,Transaction_Frequency,Default_Risk,Loan_Approval_Status
0,1,Female,25,Married,2,Graduate,Employed,Business,Own,Urban,...,24535,209,Home,4.27,Secured,Yes,8,20,0.81,1
1,2,Male,36,Married,2,High School,Employed,Business,Own,Suburban,...,8288,33,Home,14.78,Unsecured,Yes,9,9,0.17,0
2,3,Male,43,Single,0,Postgraduate,Self-Employed,Freelancer,Own,Urban,...,10308,159,Vehicle,12.33,Secured,Yes,7,27,0.25,0
3,4,Female,28,Married,0,High School,Self-Employed,Freelancer,Rent,Suburban,...,33937,39,Personal,8.77,Secured,No,9,16,0.27,1
4,5,Female,32,Single,0,Graduate,Employed,Salaried,Rent,Suburban,...,23360,34,Home,9.04,Unsecured,No,1,17,0.32,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51995,51996,Female,47,Divorced,1,High School,Self-Employed,Professional,Own,Urban,...,15723,60,Home,12.52,Secured,Yes,2,26,0.61,1
51996,51997,Male,25,Married,3,High School,Self-Employed,Freelancer,Rent,Urban,...,21209,237,Vehicle,5.21,Unsecured,No,8,26,0.04,1
51997,51998,Female,48,Married,0,High School,Unemployed,Business,Rent,Urban,...,10540,60,Education,10.60,Unsecured,No,8,24,0.31,0
51998,51999,Female,48,Divorced,1,High School,Employed,Business,Own,Urban,...,16765,32,Vehicle,8.11,Secured,Yes,3,12,0.34,0


In [75]:
df_loans = df_loans.drop(columns=['City/Town','Monthly_Expenses','Existing_Loans','Total_Existing_Loan_Amount','Outstanding_Debt','Loan_History','Loan_Amount_Requested','Loan_Term',
                                 'Interest_Rate','Loan_Type','Co-Applicant','Default_Risk'])
df_loans

,Applicant_ID,Gender,Age,Marital_Status,Dependents,Education,Employment_Status,Occupation_Type,Residential_Status,Annual_Income,Credit_Score,Loan_Purpose,Bank_Account_History,Transaction_Frequency,Loan_Approval_Status
0,1,Female,25,Married,2,Graduate,Employed,Business,Own,139901,743,Home,8,20,1
1,2,Male,36,Married,2,High School,Employed,Business,Own,21162,468,Home,9,9,0
2,3,Male,43,Single,0,Postgraduate,Self-Employed,Freelancer,Own,27815,389,Vehicle,7,27,0
3,4,Female,28,Married,0,High School,Self-Employed,Freelancer,Rent,137853,778,Personal,9,16,1
4,5,Female,32,Single,0,Graduate,Employed,Salaried,Rent,81753,752,Home,1,17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51995,51996,Female,47,Divorced,1,High School,Self-Employed,Professional,Own,56085,381,Home,2,26,1
51996,51997,Male,25,Married,3,High School,Self-Employed,Freelancer,Rent,79062,726,Vehicle,8,26,1
51997,51998,Female,48,Married,0,High School,Unemployed,Business,Rent,57471,396,Education,8,24,0
51998,51999,Female,48,Divorced,1,High School,Employed,Business,Own,49495,362,Vehicle,3,12,0


In [76]:
import sdv

metadata = sdv.metadata.Metadata.detect_from_dataframe(data=df_loans)
synthesizer = sdv.single_table.GaussianCopulaSynthesizer(metadata)
synthesizer.fit(df_loans)
synthetic_data = synthesizer.sample(num_rows=100)
synthetic_data

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,Applicant_ID,Gender,Age,Marital_Status,Dependents,Education,Employment_Status,Occupation_Type,Residential_Status,Annual_Income,Credit_Score,Loan_Purpose,Bank_Account_History,Transaction_Frequency,Loan_Approval_Status
0,13040044,Male,57,Single,0,Graduate,Self-Employed,Freelancer,Own,113657,845,Personal,6,20,1
1,9710440,Male,41,Married,2,Graduate,Employed,Freelancer,Rent,93389,468,Personal,3,27,0
2,9408792,Male,36,Married,1,High School,Employed,Salaried,Rent,48039,846,Home,4,16,1
3,5432035,Female,36,Single,1,Graduate,Unemployed,Salaried,Rent,101647,655,Home,6,24,0
4,559579,Female,39,Married,3,Postgraduate,Self-Employed,Salaried,Rent,58074,444,Education,6,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,11251168,Female,48,Single,2,High School,Unemployed,Freelancer,Rent,89262,558,Personal,3,28,1
96,14137870,Female,41,Married,1,High School,Self-Employed,Freelancer,Own,56953,368,Vehicle,1,18,0
97,16202018,Female,34,Married,3,Postgraduate,Unemployed,Salaried,Own,127737,519,Education,3,10,0
98,5575336,Female,25,Married,2,High School,Unemployed,Business,Own,63761,849,Education,7,15,1


In [77]:
synthesizer.get_learned_distributions()

{'Gender': {'distribution': 'beta',
  'learned_parameters': {'loc': 7.109038555876352e-06,
   'scale': 0.9999814237531197,
   'a': 1.0059178076915072,
   'b': 1.003942319822961}},
 'Age': {'distribution': 'beta',
  'learned_parameters': {'loc': 14.178538243367676,
   'scale': 116.45852650969424,
   'a': 4.121679608003936,
   'b': 15.294712060639306}},
 'Marital_Status': {'distribution': 'beta',
  'learned_parameters': {'loc': 5.1226860836209546e-05,
   'scale': 0.9998343994833745,
   'a': 0.998543299680159,
   'b': 0.9948803775691382}},
 'Dependents': {'distribution': 'beta',
  'learned_parameters': {'loc': 9.886047057660265e-06,
   'scale': 0.9999859512295073,
   'a': 1.0042413598362572,
   'b': 1.0065005036934151}},
 'Education': {'distribution': 'beta',
  'learned_parameters': {'loc': 7.551823235527928e-05,
   'scale': 0.9998941045929155,
   'a': 1.0069489759536845,
   'b': 1.0068267521208731}},
 'Employment_Status': {'distribution': 'beta',
  'learned_parameters': {'loc': 1.4024575

In [78]:
loan_only_pivot_df = pivot_df[pivot_df['Loans'] == 1.0]
loan_only_pivot_df

,Customer code,Employment,Gender,age,Residence,Foreigner,Active,Pension,Income,Segment,Accounts,Investments,Credit Cards,Loans,Short Term Deposit,Medium Term Deposit,Long Term Deposit
15,15906,N,Female,55,S,N,1.0,0.0,81005.49,2.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
21,15914,F,Female,55,S,N,1.0,0.0,331242.63,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
25,15918,B,Female,50,S,N,1.0,0.0,298795.08,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
29,15922,B,Male,61,S,N,1.0,0.0,225127.50,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
34,15927,B,Male,62,S,N,1.0,1.0,278937.57,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813419,1399438,N,Male,51,S,N,1.0,0.0,99183.06,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
869336,1464311,N,Female,90,S,N,0.0,0.0,101440.23,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
903482,1500806,N,Female,49,S,N,1.0,0.0,101440.23,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
910749,1508643,N,Male,51,S,N,1.0,0.0,101440.23,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [80]:
pivot_df['Gender'] = pivot_df['Gender'].replace(to_replace=['V','H'], value=['Male','Female'])
loan_only_pivot_df = pivot_df[pivot_df['Loans'] == 1.0]
pivot_loan_info_df = pd.DataFrame(columns=df_loans.columns)
print((df_loans['Annual_Income'].min(), df_loans['Annual_Income'].max()))
temp_scaler = MinMaxScaler(feature_range=(df_loans['Annual_Income'].min(), df_loans['Annual_Income'].max()))
loan_only_pivot_df['Income'] = temp_scaler.fit_transform(loan_only_pivot_df['Income'].to_numpy().reshape(-1,1))
loan_only_pivot_df

(20009, 149998)


,Customer code,Employment,Gender,age,Residence,Foreigner,Active,Pension,Income,Segment,Accounts,Investments,Credit Cards,Loans,Short Term Deposit,Medium Term Deposit,Long Term Deposit
15,15906,N,Female,55,S,N,1.0,0.0,20807.440611,2.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
21,15914,F,Female,55,S,N,1.0,0.0,23660.645896,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
25,15918,B,Female,50,S,N,1.0,0.0,23290.678747,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
29,15922,B,Male,61,S,N,1.0,0.0,22450.720584,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
34,15927,B,Male,62,S,N,1.0,1.0,23064.263306,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813419,1399438,N,Male,51,S,N,1.0,0.0,21014.701367,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
869336,1464311,N,Female,90,S,N,0.0,0.0,21040.437632,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
903482,1500806,N,Female,49,S,N,1.0,0.0,21040.437632,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
910749,1508643,N,Male,51,S,N,1.0,0.0,21040.437632,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [83]:
for index, row in loan_only_pivot_df.iterrows():
    temp_df = synthesizer.sample(num_rows=1)
    temp_df['Applicant_ID'] = row['Customer code']
    temp_df['Gender'] = row['Gender']
    temp_df['Annual_Income'] = row['Income']
    pivot_loan_info_df = pd.concat([pivot_loan_info_df, temp_df])
pivot_loan_info_df

<ipython-input-83-be48d58ba86c>:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pivot_loan_info_df = pd.concat([pivot_loan_info_df, temp_df])


,Applicant_ID,Gender,Age,Marital_Status,Dependents,Education,Employment_Status,Occupation_Type,Residential_Status,Annual_Income,Credit_Score,Loan_Purpose,Bank_Account_History,Transaction_Frequency,Loan_Approval_Status
0,15906,Female,45,Single,2,Postgraduate,Employed,Professional,Rent,20807.440611,849,Vehicle,8,13,1
0,15914,Female,30,Married,1,Postgraduate,Self-Employed,Business,Own,23660.645896,383,Personal,2,7,1
0,15918,Female,53,Single,1,Postgraduate,Employed,Freelancer,Own,23290.678747,448,Vehicle,4,9,0
0,15922,Male,29,Married,2,Graduate,Unemployed,Freelancer,Other,22450.720584,472,Education,9,25,0
0,15927,Male,55,Married,1,High School,Self-Employed,Business,Rent,23064.263306,665,Personal,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1399438,Male,34,Married,3,High School,Unemployed,Professional,Own,21014.701367,300,Home,1,28,0
0,1464311,Female,55,Married,2,Graduate,Employed,Freelancer,Other,21040.437632,820,Home,4,24,1
0,1500806,Female,31,Single,3,High School,Self-Employed,Professional,Own,21040.437632,641,Personal,3,11,0
0,1508643,Male,48,Married,0,Postgraduate,Employed,Freelancer,Own,21040.437632,363,Home,2,20,0


In [84]:
pivot_loan_info_df.to_csv("pivot_loan_info.csv")
pivot_df.to_csv("pivot.csv")